In [1]:
!pip install segmentation-models-pytorch
!pip install torchinfo

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16425 sha256=973c57d0de7212f701ea66c8f90fe19e9e387f789c6e11e451ec893bf8f85b8a
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=d74d9df1783867c9910d9b23b79dfa0f9509a9d7b66dad60947a0d211fbd3ee8
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientne

In [2]:
import os
import random
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score
import logging
from pathlib import Path
import shutil
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2

from tqdm.auto import tqdm

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss, JaccardLoss

# Constants
IMAGE_SIZE = (224, 224)
SUPPORTED_FORMATS = ['.jpg', '.jpeg', '.png']

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
class Block(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3,stride=1,padding=1)
        self.relu  = nn.ReLU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3,stride=1,padding=1)

    def forward(self, x):
        return self.relu(self.conv2(self.relu(self.conv1(x)))).to(device)


class Encoder(nn.Module):
    def __init__(self, chs=(3,64,128,256,512,1024)):
        super().__init__()
        self.enc_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)])
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        ftrs = []
        for block in self.enc_blocks:
            x = block(x)
            ftrs.append(x)
            x = self.pool(x)
        return ftrs


class Decoder(nn.Module):
    def __init__(self, chs=(1024, 512, 256, 128, 64)):
        super().__init__()
        self.chs  = chs
        self.upconvs  = nn.ModuleList([nn.ConvTranspose2d(chs[i], chs[i+1], 2, 2) for i in range(len(chs)-1)])
        self.dec_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)])

    def forward(self, x, encoder_features):
        for i in range(len(self.chs)-1):
            x  = self.upconvs[i](x)
            enc_ftrs = self.crop(encoder_features[i], x)
            x = torch.cat([x, enc_ftrs], dim=1)
            x = self.dec_blocks[i](x)
        return x

    def crop(self, enc_ftrs, x):
        _, _, H, W = x.shape
        enc_ftrs = transforms.CenterCrop([H, W])(enc_ftrs)
        return enc_ftrs


class UNet(nn.Module):
    def __init__(self, enc_chs=(3,64,128,256,512,1024), dec_chs=(1024, 512, 256, 128, 64), num_class=1):
        super().__init__()
        self.encoder = Encoder(enc_chs)
        self.decoder  = Decoder(dec_chs)
        self.head  = nn.Conv2d(dec_chs[-1], num_class, 1)


    def forward(self, x):
        enc_ftrs = self.encoder(x)
        out = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        out = self.head(out)
        return out

In [4]:
import torch
import torch.nn.functional as F
import numpy as np
from torchvision import transforms
from PIL import Image

class MaskGenerator:
    def __init__(self, model, device, input_size=(256, 256)):
        """
        Initialize the mask generator with a pre-trained UNet model
        
        Args:
            model: Pre-trained UNet model
            device: torch.device to use for computation
            input_size: Tuple of (height, width) for input image resizing
        """
        self.model = model
        self.device = device
        self.input_size = input_size
        
        self.transform = transforms.Compose([
            transforms.Resize(input_size),
            transforms.ToTensor(),
        ])
    
    def preprocess_image(self, image):
        """
        Preprocess the input image for the model
        
        Args:
            image: PIL Image or numpy array
            
        Returns:
            torch.Tensor: Preprocessed image tensor
        """
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        
        # Apply transformations
        image_tensor = self.transform(image)
        return image_tensor.unsqueeze(0).to(self.device)  # Add batch dimension and move to device
    
    def postprocess_mask(self, mask_tensor):
        """
        Postprocess the mask tensor to get the final binary mask
        
        Args:
            mask_tensor: Tensor output from the model
            
        Returns:
            numpy.ndarray: Binary mask array
        """
        # Convert to numpy and ensure proper dimensions
        mask = mask_tensor.squeeze(0).cpu().detach().permute(1, 2, 0).numpy()
        
        # Apply thresholding to get binary mask
        mask[mask < 0] = 0
        mask[mask > 0] = 1
        
        return mask

    def generate_mask(self, image):
        """
        Generate binary mask for the input image using the UNet model
        
        Args:
            image: PIL Image or numpy array
            
        Returns:
            numpy.ndarray: Binary mask array
        """
        # Ensure model is in eval mode
        self.model.eval()
        
        # Preprocess image
        image_tensor = self.preprocess_image(image)
        
        with torch.no_grad():
            # Get model prediction
            mask_tensor = self.model(image_tensor)
            
            # Postprocess the mask
            mask = self.postprocess_mask(mask_tensor)
            
        return mask
    
    def generate_batch_masks(self, images):
        """
        Generate masks for a batch of images
        
        Args:
            images: List of PIL Images or numpy arrays
            
        Returns:
            List[numpy.ndarray]: List of mask arrays
        """
        return [self.generate_mask(img) for img in images]

In [5]:
class ImagePreprocessor:
    """Skin Cancer Image preprocessing pipeline"""
    
    @staticmethod
    def hair_remove(image):
        """Remove hair from skin images"""
        try:
            grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 17))
            blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
            _, threshold = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
            final_image = cv2.inpaint(image, threshold, 1, cv2.INPAINT_TELEA)
            return final_image
        except Exception as e:
            print(f"Error in hair removal: {str(e)}")
            return image

    @staticmethod
    def sharpen_image(image):
        """Sharpen image using unsharp masking"""
        gaussian = cv2.GaussianBlur(image, (0, 0), 2.0)
        return cv2.addWeighted(image, 1.5, gaussian, -0.5, 0)

In [6]:
def preprocess_image(image, target_size=(224, 224)):
    """Apply all preprocessing steps to an image"""
    preprocessor = ImagePreprocessor()
    
    image = preprocessor.hair_remove(image)
    image = preprocessor.sharpen_image(image)
    image = cv2.resize(image, target_size, interpolation=cv2.INTER_NEAREST)
    
    return image

In [7]:
def process_and_organize_dataset(source_path, destination_path, model, device):
    """Process images and organize them into the new structure"""
    source_path = Path(source_path)
    destination_path = Path(destination_path)
    
    mask_generator = MaskGenerator(model, device)
    
    splits = ['train_directory', 'test_directory', 'validation_directory']
    for split in splits:
        split_path = source_path / split
        dest_split = split
        
        for category in ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']:
            category_path = split_path / category
            if not category_path.exists():
                continue
                
            print(f"Processing {split}/{category}...")
            
            # Process each image in the category
            for img_file in tqdm([f for ext in SUPPORTED_FORMATS for f in category_path.glob(f'*{ext}')]):
                img = cv2.imread(str(img_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                # Apply preprocessing
                processed_img = preprocess_image(img.copy(), target_size=(224, 224))
                
                # Generate filename without extension
                filename = img_file.stem
                
                # Create and save ground truth mask from preprocessed image
                mask = mask_generator.generate_mask(processed_img)
                if processed_img.shape[:2] != mask.shape[:2]:
                    mask = cv2.resize(mask, processed_img.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)
                if len(mask.shape) == 2:
                    mask = np.expand_dims(mask, axis=2)
                mask_save = (mask * 255).astype(np.uint8)
                mask_path = destination_path / 'ground_truth' / dest_split / category / f"{filename}.jpg"
                cv2.imwrite(str(mask_path), mask_save)
                
                # Create and save segmented image
                segmented = processed_img.copy()
                segmented[mask[:, :, 0] == 0] = 0
                segmented_path = destination_path / 'unet_segmented' / dest_split / category / f"{filename}.jpg"
                cv2.imwrite(str(segmented_path), cv2.cvtColor(segmented, cv2.COLOR_RGB2BGR))

In [8]:
base_directory = '/kaggle/working/unet_segmented_images'
os.mkdir(base_directory)

subfolders = ['ground_truth', 'unet_segmented']
directory = ['train_directory', 'test_directory', 'validation_directory']
classes = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

for subf in subfolders:
    path = os.path.join(base_directory, subf)
    os.mkdir(path)
    for dirc in directory:
        path = os.path.join(base_directory, subf, dirc)
        os.mkdir(path)
        for cls in classes:
            path = os.path.join(base_directory, subf, dirc, cls)
            os.mkdir(path)

In [9]:
source_path = "/kaggle/input/multiclassskincancer"
destination_path = "/kaggle/working/unet_segmented_images"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
unet_model = UNet(enc_chs=(3,32, 64,128,256), dec_chs=(256, 128, 64, 32), num_class=1)
model_path = '/kaggle/input/image-segmentation-unet-model/segmentation_model.pth'
unet_model.load_state_dict(torch.load(model_path,  map_location=torch.device('cpu')))
unet_model.eval()
unet_model.to(device)

<ipython-input-9-ea199026fbea>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet_model.load_state_dict(torch.load(model_path,  map_location=torch.device('cpu')))


UNet(
  (encoder): Encoder(
    (enc_blocks): ModuleList(
      (0): Block(
        (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU()
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1): Block(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (2): Block(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU()
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (3): Block(
        (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU()
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, 

In [11]:
process_and_organize_dataset(source_path, destination_path, unet_model, device)

Processing train_directory/nv...


  0%|          | 0/5115 [00:00<?, ?it/s]

Processing train_directory/mel...


  0%|          | 0/5950 [00:00<?, ?it/s]

Processing train_directory/bkl...


...

  0%|          | 0/24 [00:00<?, ?it/s]

Processing validation_directory/vasc...


  0%|          | 0/10 [00:00<?, ?it/s]

Processing validation_directory/df...


  0%|          | 0/6 [00:00<?, ?it/s]

In [13]:
base_dir = '/kaggle/working/unet_segmented_images'
subfolders = ['ground_truth', 'unet_segmented']
directory = ['train_directory', 'test_directory', 'validation_directory']
classes = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

for subf in subfolders:
    print(subf)
    for dirc in directory:
        print(dirc)
        for cls in classes:
            path = os.path.join(base_dir, subf, dirc, cls)
            print(f"{cls}    : ", len(os.listdir(path)))

ground_truth
train_directory
nv    :  5115
mel    :  5950
bkl    :  5990
bcc    :  5462
akiec    :  5510
vasc    :  4810
df    :  4090
test_directory
nv    :  883
mel    :  46
bkl    :  88
bcc    :  35
akiec    :  30
vasc    :  13
df    :  8
validation_directory
nv    :  707
mel    :  37
bkl    :  71
bcc    :  28
akiec    :  24
vasc    :  10
df    :  6
unet_segmented
train_directory
nv    :  5115
mel    :  5950
bkl    :  5990
bcc    :  5462
akiec    :  5510
vasc    :  4810
df    :  4090
test_directory
nv    :  883
mel    :  46
bkl    :  88
bcc    :  35
akiec    :  30
vasc    :  13
df    :  8
validation_directory
nv    :  707
mel    :  37
bkl    :  71
bcc    :  28
akiec    :  24
vasc    :  10
df    :  6
